In [2]:
import pandas as pb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import os.path
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
import gzip

from saxpy.znorm import znorm
from saxpy.sax import ts_to_string
from saxpy.alphabet import cuts_for_asize

In [3]:
X_L = []
X_R = []
Y = []

for root, dirs, files in os.walk('Recording Data'):
    for file in files:
        if file.endswith(".txt") and file != 'Subject codes.txt':
            file_path = os.path.join(root, file)
            match root[-1]:
                case '1' | '2':
                    has_dyslexia = 1
                case _:
                    has_dyslexia = 0
            data = pd.read_csv(file_path, sep='\t', index_col=None, decimal=",")
            data['r_dist'] = ts_to_string(znorm(np.array((data['RX'].diff() ** 2 + data['RY'].diff() ** 2) ** 0.5)), cuts_for_asize(10))
            data['l_dist'] = ts_to_string(znorm(np.array((data['LX'].diff() ** 2 + data['LY'].diff() ** 2) ** 0.5)), cuts_for_asize(10))
            X_L.append(data['l_dist'][1:])
            X_R.append(data['r_dist'][1:])
            Y.append(has_dyslexia)

In [4]:
X = X_L + X_R
Y = Y * 2

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [6]:
k_m = [1, 3, 4, 5, 15, 20, 34, 35, 36]

In [10]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

f_scores = {}
for k in k_m:
    f_scores[k] = [[], []]

for train_index, test_index in kf.split(X):
    X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
    Y_train, Y_test = [Y[i] for i in train_index], [Y[i] for i in test_index]
    for k in k_m:
        predicts = []
        for x1 in X_test:
            x1 = np.array(x1)
            Cx1 = len(gzip.compress(x1))
            distance_from_x1 = []
            for x2 in X_train:
                x2 = np.array(x2)
                Cx2 = len(gzip.compress(x2))
                x1x2 = np.concatenate((x1, x2))
                Cx1x2 = len(gzip.compress(np.array(x1x2)))
                ncd = (Cx1x2 - min(Cx1 , Cx2)) / max (Cx1 , Cx2 )
                distance_from_x1.append(ncd)
                        
            sorted_idx = np.argsort(np.array(distance_from_x1))
            top_k_class = np.array(Y_train)[sorted_idx[:k]].tolist()
            predict_class = max(set(top_k_class), key = top_k_class.count)
            predicts.append(predict_class)

        sensitivity, specificity = calculate_sensitivity_specificity(Y_test, predicts)
        f1 = classification_report(Y_test, predicts, output_dict=True)['macro avg']['f1-score']
        f_scores[k][0].append(f1)
        f_scores[k][1].append([sensitivity, specificity])

In [11]:
sensitivity = {k: sum([scores[0] for scores in f_scores[k][1]]) / len([scores[0] for scores in f_scores[k][1]]) for k in k_m}
specificity = {k: sum([scores[1] for scores in f_scores[k][1]]) / len([scores[1] for scores in f_scores[k][1]]) for k in k_m}
data = [f_scores[k][0] + [sum(f_scores[k][0]) / len(f_scores[k][0]), sensitivity[k], specificity[k]] for k in k_m]

In [12]:
table = pd.DataFrame(data, index=k_m, columns=['1_split', '2_split', '3_split', '4_split', '5_split', 'avg_score', 'sensitivity', 'specificity'])

In [13]:
table

,1_split,2_split,3_split,4_split,5_split,avg_score,sensitivity,specificity
1,0.783626,0.739388,0.702485,0.837719,0.824035,0.777451,0.725495,0.843608
3,0.837719,0.727941,0.783626,0.864469,0.851107,0.812972,0.773842,0.858892
4,0.783150,0.639160,0.715749,0.851107,0.809559,0.759745,0.670730,0.864953
5,0.864469,0.783626,0.810256,0.877821,0.864766,0.840188,0.825365,0.858892
15,0.891176,0.783626,0.877280,0.904551,0.905388,0.872404,0.882264,0.864774
20,0.891176,0.783626,0.863971,0.904551,0.891892,0.867043,0.871550,0.864774
34,0.891575,0.824292,0.877280,0.891176,0.891892,0.875243,0.882461,0.870835
35,0.904972,0.824292,0.890613,0.891176,0.891892,0.880589,0.892339,0.870835
36,0.904972,0.824292,0.877280,0.891176,0.878356,0.875215,0.881625,0.870835


In [7]:
def calculate_sensitivity_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return sensitivity, specificity